In [1]:
import pandas as pd
import numpy as np
import sklearn.linear_model as SkLinear
import joblib as jb

pd.reset_option('all')
pd.options.mode.chained_assignment = None

In [2]:
def severalAppend(matrice, ElementToAdd):
    for i in ElementToAdd:
        matrice.append(i)
    return matrice

def MatchColumnAndText(Pandasmatrice, EndText):
    for i in range(0,len(Pandasmatrice.columns)):
        if Pandasmatrice.columns[i][len(Pandasmatrice.columns[i])-len(EndText):] == (EndText):
            print(f'{i} : {Pandasmatrice.columns[i]}')

def AddPreText(MatriceLigne,PreText):
    ligne = []
    for i in MatriceLigne:
        text = PreText + i
        ligne.append(text)
    return ligne

def GetTheBeta(X,Y):
    Y = np.array(Y)
    X= np.array(X)
    BetaModel = SkLinear.LinearRegression()
    BetaModel.fit(X,Y)
    return [BetaModel.intercept_,BetaModel.coef_]

In [3]:
YourPath = 'PATH'

In [4]:
modele = jb.load('LogisticModelV2.joblib')
data = pd.read_excel(YourPath+'\Test\Z_DataTest_All_Clean.xlsx')

In [5]:
Model_Data_Y = data[data.columns[[1]]]

In [6]:
loc = [9,32,86]
loc = severalAppend(loc, [27,54,108,81,135])
VoixEuro = AddPreText(data.columns[[0,23,24,25,26]],"Log_")[1:]
VoixLegi = AddPreText(data.columns[[0,50,51,52,53]],"Log_")[1:]
VoixPres = AddPreText(data.columns[[0,104,105,106,107]],"Log_")[1:]
Voix = [VoixEuro,VoixLegi,VoixPres]
Matrice_Voix = data[data.columns[[0,23,24,25,26,50,51,52,53,104,105,106,107]]]

In [7]:
for arrayVoix in Voix:
    for colonne in arrayVoix:
        variable = colonne[4:]
        for ligne in range(0,len(Matrice_Voix)):
            Matrice_Voix.loc[ligne,colonne] =  np.log(Matrice_Voix.loc[ligne,variable]+0.85)
            
Matrice_Voix = Matrice_Voix.drop(Matrice_Voix.columns[:int(len(Matrice_Voix.columns)/2)+1],axis=1)

In [8]:
Matrice_ID = pd.DataFrame(data[data.columns[0]])
Matrice_ID["Zone"] = Matrice_ID["ID"]
for i in range(0,len(Matrice_ID)):
    Matrice_ID.loc[i,"Zone"] = Matrice_ID["Zone"][i][:2]
Matrice_ID = pd.get_dummies(Matrice_ID,columns=['Zone'],dtype=int)
base = pd.DataFrame(data[data.columns[0]])
for cat in range(0,len(Voix)):
    base = base.join(Matrice_Voix[Voix[cat]].rank(axis=1,ascending=False))

Voixranktemp = ["_1er","_2em","_3em","_4em"]
Voixrank = [AddPreText(Voixranktemp,"Euro"),AddPreText(Voixranktemp,"Legi"),AddPreText(Voixranktemp,"Pres")]

for var in range(1,base.shape[1]):
    temp = base.columns[var]
    base = base.rename(columns={temp:Voixrank[int((var-1)/4)][(var-1)%4]})
    

In [9]:
X = [[1],[2],[3],[4]]
Linear = ['Intercept','Beta']
Scrutin = ['Euro','Legi','Pres']

a = []
for i in range(0,len(Scrutin)):
    for y in range(0,len(Linear)):
        a.append(Scrutin[i]+"_"+Linear[y])
base[a] = 0

In [10]:
matrice = []
for cat in range(0,len(Voixrank)):
    temp = []
    for i in range(0,len(base)):
        inter = np.array(base[Voixrank[cat]].iloc[i]).reshape(4,1)
        intercep, beta = GetTheBeta(X,inter)
        intercep, beta = intercep[0],beta[0][0]
        temp.append([intercep, beta])
    matrice.append(temp)

In [11]:
for i in range(0,len(a)):
    base[a[i]] = pd.DataFrame(matrice[int((i)/2)])[i%2]
base = base.drop(['ID'],axis=1)

In [12]:
Model_Data = Matrice_ID

InscritsVariation = data[data.columns[[3,30,84]]]
column = InscritsVariation.columns
InscritsVariation["VarInscritEuroLegi"] = np.log(InscritsVariation[column[0]]/InscritsVariation[column[1]])
InscritsVariation["VarInscritLegiPres"] = np.log(InscritsVariation[column[1]]/InscritsVariation[column[2]])

In [13]:
Modele_data_tmp = InscritsVariation.join(base)
Modele_data_tmp = Modele_data_tmp.join(data[data.columns[loc]])
Modele_data_tmp_columns = Modele_data_tmp.columns


In [14]:
from sklearn.preprocessing import StandardScaler

CentrerReduire = StandardScaler()
CentrerReduire.fit(Modele_data_tmp)
Modele_data_tmp = CentrerReduire.transform(Modele_data_tmp) 
Modele_data_tmp = pd.DataFrame(Modele_data_tmp,columns=Modele_data_tmp_columns)
Model_Data = Model_Data.join(Modele_data_tmp)

In [15]:
Model_Data 

,ID,Zone_01,Zone_02,Zone_03,Zone_04,Zone_05,Zone_06,Zone_07,Zone_08,Zone_09,...,Pres_Intercept,Pres_Beta,Euro2024_% Abs/Ins,LegiT12022_% Abs/Ins,PresT12022_% Abs/Ins,Euro2024_Voix/Inscrits_ecarttype,LegiT12022_Voix/Inscrits_ecarttype,PresT12022_Voix/Inscrits_ecarttype,LegiT22022_Voix/Inscrits_ecarttype,PresT22022_Voix/Inscrits_ecarttype
0,01_01,1,0,0,0,0,0,0,0,0,...,1.155468,-1.155468,-0.418600,-0.232671,-0.480894,0.375325,-1.668185,-0.487579,0.396693,-0.014583
1,01_02,1,0,0,0,0,0,0,0,0,...,1.155468,-1.155468,-0.544125,-0.281030,-0.651982,0.469243,-1.143456,-0.077572,-0.075718,0.084162
2,01_03,1,0,0,0,0,0,0,0,0,...,0.274818,-0.274818,0.145285,0.471226,-0.129387,-0.737863,-0.807936,-0.716316,-0.653627,-0.047326
3,01_04,1,0,0,0,0,0,0,0,0,...,1.155468,-1.155468,-0.377412,-0.063413,-0.551403,0.868618,-1.281330,0.062365,-0.502928,0.039364
4,01_05,1,0,0,0,0,0,0,0,0,...,0.715143,-0.715143,-0.149897,-0.032517,-0.207154,0.386629,0.536297,-0.386549,-0.510123,-0.445874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,ZZ_07,0,0,0,0,0,0,0,0,0,...,-1.046157,1.046157,3.134362,2.455303,3.174160,-1.971701,-0.254410,-0.291069,-1.483136,0.238071
568,ZZ_08,0,0,0,0,0,0,0,0,0,...,0.274818,-0.274818,4.185638,4.753715,5.757065,-3.339255,-0.991770,-3.179388,-2.756165,-4.301185
569,ZZ_09,0,0,0,0,0,0,0,0,0,...,-1.486481,1.486481,3.681575,4.420573,4.233867,-2.424405,-1.064794,-1.569485,-3.317975,-2.512678
570,ZZ_10,0,0,0,0,0,0,0,0,0,...,-1.046157,1.046157,3.406007,3.774438,3.613804,-2.893963,-0.812580,-1.680154,-2.886029,-2.072462


In [16]:
Model_Data_X = Model_Data.drop(Model_Data.columns[0],axis=1)
Model_Data_Columns = Model_Data.columns

In [17]:
DDAATTAA = Model_Data_Y.join(Model_Data_X)
DDAATTAA = pd.DataFrame(data[data.columns[0]]).join(DDAATTAA)
DDAATTAA = DDAATTAA.sample(frac=1,axis=0).reset_index()
Model_Data_Y = np.array(DDAATTAA[DDAATTAA.columns[2]])
Model_Data_X = np.array(DDAATTAA[DDAATTAA.columns[3:]])

In [18]:
Prediction = modele.predict(Model_Data_X)

In [19]:
resultatNet = [0,0,0,0]
resultatrelatif = [0,0,0,0]
Bloc = ["UP", "C", "D", "ED"]
count = 0
for i in Prediction:
    resultatNet[i-1]+=1
    count +=1
resultatNet
for i in range(0,len(resultatNet)):
    resultatrelatif[i] = resultatNet[i]/count

for i in range(0,4):
    print(f"Parts des votes pour {Bloc[i]}: {str(resultatrelatif[i]*100)[:6]}%\n")

Parts des votes pour UP: 25.174%

Parts des votes pour C: 45.104%

Parts des votes pour D: 6.8181%

Parts des votes pour ED: 22.902%



Parts des votes pour UP: 25.174%

Parts des votes pour C: 45.104%

Parts des votes pour D: 6.8181%

Parts des votes pour ED: 22.902%